In [1]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain import OpenAI, PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains.llm import LLMChain
from langchain.chains.constitutional_ai.principles import PRINCIPLES
from langchain.chains.constitutional_ai.base import ConstitutionalChain
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory
from langchain.callbacks import get_openai_callback

import deeplake
import os
from dotenv import load_dotenv

load_dotenv()


/home/haritsahm/.virtualenvs/spotify_recsys/lib/python3.10/site-packages/deeplake/util/check_latest_version.py:32: UserWarning: A newer version of deeplake (3.8.14) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


True

In [60]:
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

my_activeloop_dataset_name = "spotify_reviews_cleaned_filtered_balanced_50K_docs_1000_chunk"
dataset_path = f"./../deeplake/{my_activeloop_dataset_name}"
db = DeepLake(dataset_path=dataset_path, embedding=embeddings, num_workers=4, read_only=True)
retriever = db.as_retriever()
retriever.search_kwargs['distance_metric'] = 'cos'
retriever.search_kwargs['fetch_k'] = 25
retriever.search_kwargs['maximal_marginal_relevance'] = True
retriever.search_kwargs['k'] = 25


Deep Lake Dataset in ./../deeplake/spotify_reviews_cleaned_filtered_balanced_50K_docs_1000_chunk already exists, loading from the storage


# Generation Only

In [61]:
template = """You're a helpful assistant, please answer the following question correctly using only the information from the provided context. Do not invent stuff and simply say that you don't know to the answer because it is irrelevent with the provided context below:
{context}

Question: {question}

Answer:
"""

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=template,
)

In [62]:
principles = ['uo-evidence-2', 'uo-implications-2', 'uo-implications-3', 'uo-reasoning-3', 'uo-utility-3', 'uo-security-1']
# principles = ['uo-evidence-3', 'uo-reasoning-3', 'uo-utility-3', 'uo-security-1']

constitutional_principles = ConstitutionalChain.get_principles(principles)

llm = OpenAI(model="text-davinci-003", temperature=0)
llm_chain = LLMChain(llm=llm, prompt=prompt)

constitutional_chain = ConstitutionalChain.from_llm(
    chain=llm_chain,
    constitutional_principles=constitutional_principles,
    llm=llm,
)

In [63]:
# user question
query = "What do users dislikes about our application?"
# query = "How to check disk usage in linux?"

# retrieve relevant chunks
docs = db.similarity_search(query)
retrieved_chunks = [doc.page_content for doc in docs]

# format the prompt
chunks_formatted = "\n\n".join(retrieved_chunks)
prompt_formatted = prompt.format(context=chunks_formatted, question=query)

# generate answer
answer = constitutional_chain.run(context=chunks_formatted, question=query)
print(answer)


Users have identified several issues with the application, such as excessive clutter in the notification bar, limited features available to premium users, intrusive ad banners, and lack of voice search for mobile. These issues should be addressed in order to improve the user experience.


# Retrieval QA

In [64]:
template = """You're a helpful assistant, please answer the following question correctly using only the information from the provided context. Do not invent stuff and simply say that you don't know to the answer because it is irrelevent with the provided context below:
{context}

Question: {question}

Answer:
"""

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=template,
)

In [65]:
# llm = OpenAI(model="text-davinci-003", temperature=0)
llm = ChatOpenAI(model='gpt-3.5-turbo-1106')
# memory=ConversationBufferWindowMemory(
#         memory_key="history",
#         k=3,
#         input_key='question',
#         # output_key='answer',
#         return_messages=True)
retrieval_qa = RetrievalQA.from_chain_type(
	llm=llm,
	chain_type="stuff",
	retriever=retriever,
    # memory=memory,
    chain_type_kwargs = {'prompt': prompt, 'verbose':True},
    verbose=True
)


In [66]:
query = "What do users dislikes about our application?"
# query = "How to check disk usage in linux?"

retrieval_qa.run(query=query)



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You're a helpful assistant, please answer the following question correctly using only the information from the provided context. Do not invent stuff and simply say that you don't know to the answer because it is irrelevent with the provided context below:
As of November 2018, a user provided a negative review with a rating of 1 stars. They mentioned: 'i hate this app very much...'. This review received 1 upvotes from others.
As of June 2016, a user provided a negative review with a rating of 1 stars. They mentioned: 'it's unbearebly stupid how many features are limited to premium. this design is a turnoff, not a way to get people to pay monthly fees'. This review received 0 upvotes from others.

As of June 2019, a user provided a negative review with a rating of 2 stars. They mentioned: 'as someone who absolutely hates excessive clutter in my n

'Based on the provided reviews, users dislike the following aspects of the application:\n1. Excessive clutter in the notification bar\n2. Limited features for free users\n3. Bugs and crashes\n4. Annoying ads and ad frequency\n5. Changes in the user interface\n6. Limited skips and restrictions on features\n7. Inconsistent song selection\n8. Slow and laggy performance\n9. Removal of features\n10. Pop-up messages and notifications\n11. Inability to log in or access the account\n12. Inability to control the app session from desktop\n13. The app being laggy and unresponsive\n14. Annoying registration process\n15. Unskippable ads and ad interruptions\n16. The app freezing and becoming unusable\n17. The app being ugly and underwhelming in design\n18. The app not functioning like the website\n19. The app becoming unusable and the removal of good features\n20. The app being buggy and glitchy'

In [67]:
query = "What was the previous question?"
# query = "How to check disk usage in linux?"

retrieval_qa.run(query=query)



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You're a helpful assistant, please answer the following question correctly using only the information from the provided context. Do not invent stuff and simply say that you don't know to the answer because it is irrelevent with the provided context below:
upvotes from others.

upvotes from others.

upvotes from others.

this, but it's time to move on.'. This review received 2 upvotes from others.

This review received 9 upvotes from others.

others.

premium.'. This review received 2 upvotes from others.

As of July 2018, a user provided a negative review with a rating of 2 stars. They mentioned: 'the last update i did was really lame. i would search a song title and the option to see only that song, playlists with it, or artists involved was gone. it was only playlists. and even worst when i clicked the playlist to add that song to one of my o

"I'm sorry, I don't have that information based on the context provided."

# Conversational QA

In [68]:
template = """You're a helpful assistant, please answer the following question correctly using only the information from the provided context. Do not invent stuff and simply say that you don't know to the answer because it is irrelevent with the provided context below:
{context}

This was our previous chat history:
{chat_history}

Question: {question}

Answer:
"""

prompt = PromptTemplate(
    input_variables=["context", "chat_history", "question"],
    template=template,
)

In [69]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.chains.conversation.memory import ConversationSummaryMemory

# create GPT3 wrapper
compressor_llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)

# create compressor for the retriever
compressor = LLMChainExtractor.from_llm(compressor_llm)
compression_retriever = ContextualCompressionRetriever(
	base_compressor=compressor,
	base_retriever=retriever
)

qa_llm = ChatOpenAI(model='gpt-3.5-turbo-1106', temperature=0)
memory=ConversationBufferWindowMemory(
        memory_key="chat_history",
        k=5,
        input_key='question',
        output_key='answer',
        return_messages=True)

conversation_qa = ConversationalRetrievalChain.from_llm(
	llm=qa_llm,
	chain_type="stuff",
    memory=memory,
	retriever=retriever,
    combine_docs_chain_kwargs={
        'prompt': prompt,
    },
    verbose=True
)

def count_tokens(chain, query):
    with get_openai_callback() as cb:
        result = chain(query, return_only_outputs=True)
    return {
        'result': result,
        'token_count': cb.total_tokens
    }

In [ ]:
query = "What do users dislikes about our application?"
# query = "How to check disk usage in linux?"
response = count_tokens(conversation_qa, {'question': query})
# response = conversation_qa(, return_only_outputs=True)

In [71]:
print(response)

{'result': {'answer': 'Users dislike the excessive clutter in the notification bar, limited features for free users, frequent ads, bugs and crashes, the new home screen design, the removal of basic features, the laggy and slow performance, and the constant need for premium subscriptions to access features.'}, 'token_count': 4660}


In [72]:
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='What do users dislikes about our application?'),
  AIMessage(content='Users dislike the excessive clutter in the notification bar, limited features for free users, frequent ads, bugs and crashes, the new home screen design, the removal of basic features, the laggy and slow performance, and the constant need for premium subscriptions to access features.')]}

In [73]:
query = "What do users likes about our application?"
# query = "How to check disk usage in linux?"

response = count_tokens(conversation_qa, {'question': query})

# response = conversation_qa({'question': query}, return_only_outputs=True)



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: What do users dislikes about our application?
Assistant: Users dislike the excessive clutter in the notification bar, limited features for free users, frequent ads, bugs and crashes, the new home screen design, the removal of basic features, the laggy and slow performance, and the constant need for premium subscriptions to access features.
Follow Up Input: What do users likes about our application?
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You're a helpful assistant, please answer the following question correctly using only the information from the provided context. Do not invent stuff and simply say that you don't know to the answer because it is irrelevent 

In [74]:
print(response)

{'result': {'answer': 'Users like the organization of the app, the wide selection of music, the ease of use, the quality of the sound, the recommendations, the variety of music genres, the dark UI design, the microphone interaction, the ability to skip ads, the option to download music, the low data consumption, the great interface, the ability to create playlists, the ability to share with family, the ability to split the bill for family accounts, the limited advertisements, the easy exploration of new music, the ability to listen to music in the background, the ability to connect with Google Plus, the ability to sign in, the ability to manage accounts and followers, the ability to block users, the ability to have user profiles, and the ability to listen to music from different decades and languages.'}, 'token_count': 4809}


In [75]:
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='What do users dislikes about our application?'),
  AIMessage(content='Users dislike the excessive clutter in the notification bar, limited features for free users, frequent ads, bugs and crashes, the new home screen design, the removal of basic features, the laggy and slow performance, and the constant need for premium subscriptions to access features.'),
  HumanMessage(content='What do users likes about our application?'),
  AIMessage(content='Users like the organization of the app, the wide selection of music, the ease of use, the quality of the sound, the recommendations, the variety of music genres, the dark UI design, the microphone interaction, the ability to skip ads, the option to download music, the low data consumption, the great interface, the ability to create playlists, the ability to share with family, the ability to split the bill for family accounts, the limited advertisements, the easy exploration of new music, the ability to lis

In [76]:
query = "What was my first question?"
# query = "How to check disk usage in linux?"

count_tokens(conversation_qa, {'question': query})



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: What do users dislikes about our application?
Assistant: Users dislike the excessive clutter in the notification bar, limited features for free users, frequent ads, bugs and crashes, the new home screen design, the removal of basic features, the laggy and slow performance, and the constant need for premium subscriptions to access features.
Human: What do users likes about our application?
Assistant: Users like the organization of the app, the wide selection of music, the ease of use, the quality of the sound, the recommendations, the variety of music genres, the dark UI design, the microphone interaction, the ability to skip ads, the option to download music, the low data consumption, the great interface, the ability to create playlists, the ability to share wi

{'result': {'answer': "I'm sorry, I cannot answer that question as it is not relevant to the provided context."},
 'token_count': 3069}

In [77]:
query = "Based on the user reviews, what do we need to do to improve our application?"
# query = "How to check disk usage in linux?"

count_tokens(conversation_qa, {'question': query})



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: What do users dislikes about our application?
Assistant: Users dislike the excessive clutter in the notification bar, limited features for free users, frequent ads, bugs and crashes, the new home screen design, the removal of basic features, the laggy and slow performance, and the constant need for premium subscriptions to access features.
Human: What do users likes about our application?
Assistant: Users like the organization of the app, the wide selection of music, the ease of use, the quality of the sound, the recommendations, the variety of music genres, the dark UI design, the microphone interaction, the ability to skip ads, the option to download music, the low data consumption, the great interface, the ability to create playlists, the ability to share wi

{'result': {'answer': "Based on user reviews, some potential improvements to the application could include:\n- Providing a change log with each update to inform users of what's changing\n- Making some options free to attract more users\n- Adding music lyrics to the app\n- Improving the user interface to be more user-friendly\n- Offering a better distribution of how to access the library and improving the speed of showing results\n- Addressing bugs and crashes, and improving overall performance\n- Providing a customizable UI to allow users to personalize their experience\n- Offering a download button for offline listening\n- Improving the sound quality and refining the music recommendations\n- Adding a karaoke feature\n- Providing a free trial option for users\n- Fixing issues with reporting explicit content\n- Allowing users to access the queue\n- Addressing issues with the search functionality and improving the song updation process\n- Adding an equalizer for sound customization\n- Re

In [78]:
query = "What kind of app changes that we needed?"
# query = "How to check disk usage in linux?"

count_tokens(conversation_qa, {'question': query})



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: What do users dislikes about our application?
Assistant: Users dislike the excessive clutter in the notification bar, limited features for free users, frequent ads, bugs and crashes, the new home screen design, the removal of basic features, the laggy and slow performance, and the constant need for premium subscriptions to access features.
Human: What do users likes about our application?
Assistant: Users like the organization of the app, the wide selection of music, the ease of use, the quality of the sound, the recommendations, the variety of music genres, the dark UI design, the microphone interaction, the ability to skip ads, the option to download music, the low data consumption, the great interface, the ability to create playlists, the ability to share wi

{'result': {'answer': "Based on the user reviews, potential changes needed for the app include providing a change log with each update, making some options free to attract more users, adding music lyrics to the app, improving the user interface to be more user-friendly, offering a better distribution of how to access the library and improving the speed of showing results, addressing bugs and crashes, and improving overall performance, providing a customizable UI to allow users to personalize their experience, offering a download button for offline listening, improving the sound quality and refining the music recommendations, adding a karaoke feature, providing a free trial option for users, fixing issues with reporting explicit content, allowing users to access the queue, addressing issues with the search functionality and improving the song updation process, adding an equalizer for sound customization, reducing the number of ads and improving the advertising system, providing a more d

In [79]:
query = "How to check disk usage in linux?"

count_tokens(conversation_qa, {'question': query})



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: What do users dislikes about our application?
Assistant: Users dislike the excessive clutter in the notification bar, limited features for free users, frequent ads, bugs and crashes, the new home screen design, the removal of basic features, the laggy and slow performance, and the constant need for premium subscriptions to access features.
Human: What do users likes about our application?
Assistant: Users like the organization of the app, the wide selection of music, the ease of use, the quality of the sound, the recommendations, the variety of music genres, the dark UI design, the microphone interaction, the ability to skip ads, the option to download music, the low data consumption, the great interface, the ability to create playlists, the ability to share wi

{'result': {'answer': 'You can check disk usage in Linux by using the "df" command in the terminal. This command will display the amount of disk space used and available on the file system. Additionally, you can use the "du" command to check the disk usage of specific directories and files.'},
 'token_count': 6663}

# Conversation with Retrieval Compression

In [80]:
template = """You're a helpful assistant, please answer the following question correctly using only the information from the provided context. Do not invent stuff and simply say that you don't know to the answer because it is irrelevent with the provided context below:
{context}

This was our previous chat history:
{chat_history}

Question: {question}

Answer:
"""

prompt = PromptTemplate(
    input_variables=["context", "chat_history", "question"],
    template=template,
)

In [81]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.chains.conversation.memory import ConversationSummaryMemory

# create GPT3 wrapper
compressor_llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)

# create compressor for the retriever
compressor = LLMChainExtractor.from_llm(compressor_llm)
compression_retriever = ContextualCompressionRetriever(
	base_compressor=compressor,
	base_retriever=retriever
)

qa_llm = ChatOpenAI(model='gpt-3.5-turbo-1106', temperature=0)
memory=ConversationBufferWindowMemory(
        memory_key="chat_history",
        k=5,
        input_key='question',
        output_key='answer',
        return_messages=True)

conversation_qa = ConversationalRetrievalChain.from_llm(
	llm=qa_llm,
	chain_type="stuff",
    memory=memory,
	retriever=compression_retriever,
    combine_docs_chain_kwargs={
        'prompt': prompt,
    },
    verbose=True
)

def count_tokens(chain, query):
    with get_openai_callback() as cb:
        result = chain(query, return_only_outputs=True)
    return {
        'result': result,
        'token_count': cb.total_tokens
    }

In [82]:
query = "What do users dislikes about our application?"
# query = "How to check disk usage in linux?"
response = count_tokens(conversation_qa, {'question': query})
# response = conversation_qa(, return_only_outputs=True)

/home/haritsahm/.virtualenvs/spotify_recsys/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/haritsahm/.virtualenvs/spotify_recsys/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/haritsahm/.virtualenvs/spotify_recsys/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/haritsahm/.virtualenvs/spotify_recsys/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/haritsahm/.virtualenvs/spotify_recsys/lib/python3.10/site-packages/langchain/chain



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You're a helpful assistant, please answer the following question correctly using only the information from the provided context. Do not invent stuff and simply say that you don't know to the answer because it is irrelevent with the provided context below:
- As of November 2018, a user provided a negative review with a rating of 1 stars.
- As of June 2016, a user provided a negative review with a rating of 1 stars.
- 'i hate this app very much...'
- 'it's unbearebly stupid how many features are limited to premium. this design is a turnoff, not a way to get people to pay monthly fees'

- As of June 2019, a user provided a negative review with a rating of 2 stars.
- 'as someone who absolutely hates excessive clutter in my notification bar, this app has a tendency to drive me nuts. i don't need "spotify connections" sticky notification in addition to the "now playing" notification. as 

In [39]:
print(response)

{'result': {'answer': 'Users dislike the limited features for non-premium users, excessive clutter in the notification bar, the presence of ads and bugs in the app.'}, 'token_count': 2076}


In [ ]:
query = "What do users likes about our application?"
# query = "How to check disk usage in linux?"

response = count_tokens(conversation_qa, {'question': query})

# response = conversation_qa({'question': query}, return_only_outputs=True)

In [ ]:
query = "What was my first question?"
# query = "How to check disk usage in linux?"

count_tokens(conversation_qa, {'question': query})

# Conversation with Constitunional Principles

In [74]:
constitutional_template = """Given a text or paragraph that was generated from a response of other llm, your job is to apply transformation techniques such as summerization or grammar corrections. Only apply if needed and leave it as it is if you think it is readable enough for the reader.

{answer}
"""

constitutional_prompt = PromptTemplate(
    input_variables=["answer"],
    template=constitutional_template,
)

In [75]:
from langchain.chains import SequentialChain

principles = ['uo-evidence-2', 'uo-implications-2', 'uo-implications-3', 'uo-reasoning-3', 'uo-utility-3', 'uo-security-1']
# principles = ['uo-evidence-3', 'uo-reasoning-3', 'uo-utility-3', 'uo-security-1']

constitutional_principles = ConstitutionalChain.get_principles(principles)

qa_llm = ChatOpenAI(model='gpt-3.5-turbo-1106', temperature=0)
memory=ConversationBufferWindowMemory(
        memory_key="chat_history",
        k=5,
        # input_key='input',
        output_key='answer',
        return_messages=True)

conversation_qa = ConversationalRetrievalChain.from_llm(
	llm=qa_llm,
	chain_type="stuff",
    memory=memory,
    # memory=ConversationSummaryMemory(llm=),
	retriever=retriever,
    combine_docs_chain_kwargs={
        'prompt': prompt,
    },
    verbose=False,
    # output_keys=["answer"],
)


constitunioal_llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)
llm_chain = LLMChain(llm=constitunioal_llm, prompt=constitutional_prompt)

constitutional_chain = ConstitutionalChain.from_llm(
    chain=llm_chain,
    constitutional_principles=constitutional_principles,
    llm=constitunioal_llm,
    # output_key="refined_answer",
)

#Overall_chain
overall_chain = SequentialChain(
    chains=[conversation_qa, constitutional_chain ],
    input_variables=["question"],
    # output_variables=["refined_answer"],
    verbose=True)

In [77]:
query = "What kind of app version that makes the application rating low?"
# query = "How to check disk usage in linux?"

overall_chain.run({'question': query})



> Entering new SequentialChain chain...

> Finished chain.


'\nSome users have requested for the app to be more user-friendly and to have the option to revert back to the old version. Additionally, they have also expressed a desire for the ability to delete multiple items simultaneously.'

# TEMPLATE LEGACY

In [ ]:
# let's write a prompt for a customer support chatbot that
# answer questions using information extracted from our db
template = """You are an exceptional customer support chatbot that gently answer questions.

You know the following context information.

{chunks_formatted}

Answer to the following question from a customer. Use only information from the previous context information. Do not invent stuff.

Question: {query}

Answer:"""

prompt = PromptTemplate(
    input_variables=["chunks_formatted", "query"],
    template=template,
)

In [ ]:
# Search the database for a response based on the user's query
def search_db(user_input, db):
    print(user_input)
    retriever = db.as_retriever()
    retriever.search_kwargs['distance_metric'] = 'cos'
    retriever.search_kwargs['fetch_k'] = 100
    retriever.search_kwargs['maximal_marginal_relevance'] = True
    retriever.search_kwargs['k'] = 4
    model = ChatOpenAI(model_name='gpt-3.5-turbo')
    qa = RetrievalQA.from_llm(model, retriever=retriever, return_source_documents=True)
    return qa({'query': user_input})


In [ ]:
# the full pipeline

# user question
query = "How to check disk usage in linux?"

# retrieve relevant chunks
docs = db.similarity_search(query)
retrieved_chunks = [doc.page_content for doc in docs]

# format the prompt
chunks_formatted = "\n\n".join(retrieved_chunks)
prompt_formatted = prompt.format(chunks_formatted=chunks_formatted, query=query)

# generate answer
llm = OpenAI(model="text-davinci-003", temperature=0)
answer = llm(prompt_formatted)
print(answer)

In [32]:

print(f"{len(PRINCIPLES)} available principles")
for item in PRINCIPLES.items():
    print(item)
    print("#############\n")
# list(PRINCIPLES.items())

54 available principles
('harmful1', ConstitutionalPrinciple(critique_request='Identify specific ways in which the assistant’s last response is harmful, unethical, racist, sexist, toxic, dangerous, or illegal.', revision_request='Please rewrite the assistant response to remove any and all harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.', name='harmful1'))
#############

('harmful2', ConstitutionalPrinciple(critique_request='Explain ways in which the assistant’s preceding response may be harmful to either the human or others. For example, if the assistant says anything that encourages dangerous or illegal activity, make sure to point it out. Other examples of harm include racism, sexism, and other social biases.', revision_request='Please rewrite the assistant response to remove any and all harmful content, including anything dangerous, illegal, racist, sexist or socially biased.', name='harmful2'))
#############

('harmful3', ConstitutionalPrinciple(critique_r

In [ ]:
('uo-evidence-3', ConstitutionalPrinciple(critique_request="Discuss whether the AI model's response contains all relevant information, and does **not** contain irrelevant information.", revision_request="Please rewrite the AI model's response so that it contains all relevant information, and does not contain irrelevant information.", name='uo-evidence-3'))
#############

('uo-implications-2', ConstitutionalPrinciple(critique_request="Discuss whether the AI model's response lists appropriate suggestions for further actions or requests.", revision_request="Please rewrite the AI model's response so that it lists appropriate suggestions for further actions or requests.", name='uo-implications-2'))
#############

('uo-implications-3', ConstitutionalPrinciple(critique_request="Discuss whether the AI model's response indicates if no further actions or requests are required.", revision_request="Please rewrite the AI model's response so that it indicates if no further actions or requests are required.", name='uo-implications-3'))
#############

('uo-reasoning-3', ConstitutionalPrinciple(critique_request="Discuss whether reasoning in the AI model's response is structured (e.g. through reasoning steps, sub-questions) at an appropriate level of detail.", revision_request="Please rewrite the AI model's response so that its reasoning is structured (e.g. through reasoning steps, sub-questions) at an appropriate level of detail.", name='uo-reasoning-3'))
#############


('uo-utility-3', ConstitutionalPrinciple(critique_request="Discuss whether the AI model's response is well-formatted, e.g. free from syntactic or grammatical errors.", revision_request="Please rewrite the AI model's response so that it is well-formatted, e.g. free from syntactic or grammatical errors.", name='uo-utility-3'))
#############

('uo-security-1', ConstitutionalPrinciple(critique_request="Discuss whether the AI model's response appropriately handles incorrect, incomplete, ambiguous, or irrelevant requests or information.", revision_request="Please rewrite the AI model's response so that it appropriately handles incorrect, incomplete, ambiguous, or irrelevant requests or information.", name='uo-security-1'))
#############

